# This is the final notebook which calculates the daywise and portfolio drawdown.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [2]:
overall_result_updated = pd.read_csv('overall_result.csv')
overall_result_updated

,Date,Expiry,En_Date,Ce_En_Date,Pe_En_Date,Ce_Ex_Date,Pe_Ex_Date,Fut_En_Price,ATM_Strike,Ce_Short_Strike,...,Ce_Long_En_Price,Ce_Long_Ex_Price,Pe_Long_En_Price,Pe_Long_Ex_Price,Returns_Abs,Quantity,Lot_Size,Total_Abs_Return,Total_Capital_Return,Total_Pct_Return
0,02-01-2017,05-01-2017,02-01-2017,02-01-2017 10:30,02-01-2017 10:30,02-01-2017 15:20,02-01-2017 10:50,18067.95,18100,18100,...,16.15,10.55,32.35,58.65,-19.15,3,25,-1436.25,998563.75,-0.143625
1,03-01-2017,05-01-2017,03-01-2017,03-01-2017 10:30,03-01-2017 10:30,03-01-2017 15:20,03-01-2017 15:20,18107.00,18100,18100,...,12.10,7.75,21.60,19.50,-31.25,3,25,-2343.75,996220.00,-0.234712
2,04-01-2017,05-01-2017,04-01-2017,04-01-2017 10:30,04-01-2017 10:30,04-01-2017 15:20,04-01-2017 13:04,18048.05,18000,18000,...,6.40,4.45,6.00,3.65,34.55,3,25,2591.25,998811.25,0.260108
3,05-01-2017,05-01-2017,05-01-2017,05-01-2017 10:30,05-01-2017 10:30,05-01-2017 11:51,05-01-2017 13:56,18079.70,18100,18100,...,0.15,0.10,0.35,0.20,57.55,3,25,4316.25,1003127.50,0.432139
4,06-01-2017,12-01-2017,06-01-2017,06-01-2017 10:30,06-01-2017 10:30,06-01-2017 15:20,06-01-2017 15:20,18340.00,18300,18300,...,19.50,13.50,30.55,31.20,7.20,3,25,540.00,1003667.50,0.053832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,22-12-2017,28-12-2017,31-08-2017,22-12-2017 10:30,22-12-2017 10:30,22-12-2017 15:20,22-12-2017 15:20,25640.05,25600,25600,...,14.40,13.85,21.25,15.20,6.50,3,25,487.50,1084217.50,0.044984
242,26-12-2017,28-12-2017,01-09-2017,26-12-2017 10:30,26-12-2017 10:30,26-12-2017 15:20,26-12-2017 15:20,25611.00,25600,25600,...,5.75,6.00,10.35,7.10,9.15,3,25,686.25,1084903.75,0.063294
243,27-12-2017,28-12-2017,02-09-2017,27-12-2017 10:30,27-12-2017 10:30,27-12-2017 15:17,27-12-2017 14:23,25710.05,25700,25700,...,2.45,1.70,4.50,11.25,23.25,3,25,1743.75,1086647.50,0.160729
244,28-12-2017,28-12-2017,03-09-2017,28-12-2017 10:30,28-12-2017 10:30,28-12-2017 15:10,28-12-2017 15:15,25471.70,25500,25500,...,0.15,0.10,1.05,0.70,101.20,3,25,7590.00,1094237.50,0.698479


In [3]:
overall_result_updated['Date'] = pd.to_datetime(overall_result_updated['Date'], format='%d-%m-%Y')
overall_result_updated['Expiry'] = pd.to_datetime(overall_result_updated['Expiry'], format='%d-%m-%Y')
overall_result_updated['En_Date'] = pd.to_datetime(overall_result_updated['En_Date'], format='%d-%m-%Y')
overall_result_updated['Ce_En_Date'] = pd.to_datetime(overall_result_updated['Ce_En_Date'], format='%d-%m-%Y %H:%M')
overall_result_updated['Pe_En_Date'] = pd.to_datetime(overall_result_updated['Pe_En_Date'], format='%d-%m-%Y %H:%M')
overall_result_updated['Ce_Ex_Date'] = pd.to_datetime(overall_result_updated['Ce_Ex_Date'], format='%d-%m-%Y %H:%M')
overall_result_updated['Pe_Ex_Date'] = pd.to_datetime(overall_result_updated['Pe_Ex_Date'], format='%d-%m-%Y %H:%M')

In [24]:
overall_result_updated

,Date,Expiry,En_Date,Ce_En_Date,Pe_En_Date,Ce_Ex_Date,Pe_Ex_Date,Fut_En_Price,ATM_Strike,Ce_Short_Strike,...,Ce_Long_Ex_Price,Pe_Long_En_Price,Pe_Long_Ex_Price,Returns_Abs,Quantity,Lot_Size,Total_Abs_Return,Total_Capital_Return,Total_Pct_Return,Open
0,2017-01-02,2017-01-05,2017-01-02,2017-01-02 10:30:00,2017-01-02 10:30:00,2017-01-02 15:20:00,2017-01-02 10:50:00,18067.95,18100,18100,...,10.55,32.35,58.65,-19.15,3,25,-1436.25,998563.75,-0.143625,1000000.00
1,2017-01-03,2017-01-05,2017-01-03,2017-01-03 10:30:00,2017-01-03 10:30:00,2017-01-03 15:20:00,2017-01-03 15:20:00,18107.00,18100,18100,...,7.75,21.60,19.50,-31.25,3,25,-2343.75,996220.00,-0.234712,998563.75
2,2017-01-04,2017-01-05,2017-01-04,2017-01-04 10:30:00,2017-01-04 10:30:00,2017-01-04 15:20:00,2017-01-04 13:04:00,18048.05,18000,18000,...,4.45,6.00,3.65,34.55,3,25,2591.25,998811.25,0.260108,996220.00
3,2017-01-05,2017-01-05,2017-01-05,2017-01-05 10:30:00,2017-01-05 10:30:00,2017-01-05 11:51:00,2017-01-05 13:56:00,18079.70,18100,18100,...,0.10,0.35,0.20,57.55,3,25,4316.25,1003127.50,0.432139,998811.25
4,2017-01-06,2017-01-12,2017-01-06,2017-01-06 10:30:00,2017-01-06 10:30:00,2017-01-06 15:20:00,2017-01-06 15:20:00,18340.00,18300,18300,...,13.50,30.55,31.20,7.20,3,25,540.00,1003667.50,0.053832,1003127.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,2017-12-22,2017-12-28,2017-08-31,2017-12-22 10:30:00,2017-12-22 10:30:00,2017-12-22 15:20:00,2017-12-22 15:20:00,25640.05,25600,25600,...,13.85,21.25,15.20,6.50,3,25,487.50,1084217.50,0.044984,1083730.00
242,2017-12-26,2017-12-28,2017-09-01,2017-12-26 10:30:00,2017-12-26 10:30:00,2017-12-26 15:20:00,2017-12-26 15:20:00,25611.00,25600,25600,...,6.00,10.35,7.10,9.15,3,25,686.25,1084903.75,0.063294,1084217.50
243,2017-12-27,2017-12-28,2017-09-02,2017-12-27 10:30:00,2017-12-27 10:30:00,2017-12-27 15:17:00,2017-12-27 14:23:00,25710.05,25700,25700,...,1.70,4.50,11.25,23.25,3,25,1743.75,1086647.50,0.160729,1084903.75
244,2017-12-28,2017-12-28,2017-09-03,2017-12-28 10:30:00,2017-12-28 10:30:00,2017-12-28 15:10:00,2017-12-28 15:15:00,25471.70,25500,25500,...,0.10,1.05,0.70,101.20,3,25,7590.00,1094237.50,0.698479,1086647.50


The above code was importing the results dataframe and converting to relevant formats. Now, I will calulcate the maximum portfolio drawdown using the movement of Total_Capital_Return.

In [25]:
max_drawdown = 0
max_drawdown_index = 0
for i in range(1, len(overall_result_updated.index)):
    drawdown = (overall_result_updated['Total_Capital_Return'].iloc[i] - overall_result_updated['Total_Capital_Return'].iloc[:i].max()) / overall_result_updated['Total_Capital_Return'].iloc[:i].max()
    if drawdown < max_drawdown:
        max_drawdown = drawdown
        max_drawdown_index = i


In [26]:
max_drawdown

-0.021109268643389727

Thus, the overall drawdown = -2.11%

In [28]:
import plotly.graph_objects as go

overall_result_updated['Open'] = overall_result_updated['Total_Capital_Return'].shift(1)
overall_result_updated['Open'].iloc[0] = 1000000  # For the first day, set open as 1000000

fig = go.Figure(data=[go.Candlestick(x=overall_result_updated['Ce_En_Date'],
                                     open=overall_result_updated['Open'],
                                     high=overall_result_updated['Total_Capital_Return'],  # Use 'Total_Capital_Return' as high
                                     low=overall_result_updated['Total_Capital_Return'],   # Use 'Total_Capital_Return' as low
                                     close=overall_result_updated['Total_Capital_Return'])])

max_drawdown_text = f'Max Drawdown: {max_drawdown:.2%}'
fig.add_annotation(x=overall_result_updated['Ce_En_Date'].iloc[max_drawdown_index], 
                   y=overall_result_updated['Total_Capital_Return'].iloc[max_drawdown_index],
                   text=max_drawdown_text, showarrow=True, arrowhead=1)

fig.update_layout(title="Equity Return with Max Drawdown")

fig.show()


C:\Users\thaku\AppData\Local\Temp\ipykernel_18252\1331726719.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



This is the equity curve and drawdown, as required by the assignment. The max drawdown is annotated over the graph at the relevant position. Since graph is made using plotly, it is interactive in nature and can be scaled up and down by the viewer.

Now, I will calculate the day-wise drawdown values as required by the sample report.

In [29]:
equity_val = pd.read_csv('equity_value.csv')
equity_val

,Datetime,Equity_Value,Date
0,2017-01-02 10:30:00,1000000.00,2017-01-02
1,2017-01-02 10:31:00,999995.45,2017-01-02
2,2017-01-02 10:32:00,999996.65,2017-01-02
3,2017-01-02 10:33:00,999998.10,2017-01-02
4,2017-01-02 10:34:00,999996.05,2017-01-02
...,...,...,...
71581,2017-12-29 15:16:00,1094240.00,2017-12-29
71582,2017-12-29 15:17:00,1094241.35,2017-12-29
71583,2017-12-29 15:18:00,1094242.40,2017-12-29
71584,2017-12-29 15:19:00,1094242.20,2017-12-29


In [41]:
equity_val['Datetime'] = pd.to_datetime(equity_val['Datetime'])
equity_val['Date'] = pd.to_datetime(equity_val['Date'])
equity_val.dtypes

Datetime        datetime64[ns]
Equity_Value           float64
Date            datetime64[ns]
dtype: object

As it was noticed in the previous notebook, there were some discontinuities present in the equity_value. These are avoided here by dropping empty rows. This will also not affect the final outcome appreciably.

In [55]:
equity_val = equity_val.dropna()

In [63]:
import warnings

warnings.filterwarnings("ignore")

In [64]:
max_drawdown_df = pd.DataFrame(columns=['Date', 'Max_Drawdown', 'Drawdown'])

# iterate over each unique date
unique_dates = equity_val['Date'].unique()
unique_dates = pd.to_datetime(unique_dates)
for a in range(len(unique_dates)):
  
    curr_df = pd.DataFrame(columns=['Date', 'Max_Drawdown', 'Drawdown'])
    curr_df.loc[0, 'Date'] = unique_dates[a]
    
    date_data = equity_val[equity_val['Date'].dt.date == unique_dates[a]].reset_index(drop=True)

    price_data = date_data['Equity_Value'].reset_index(drop=True)
    
    drawdown = 0
    max_drawdown = 0
    # overall drawdown
    max_idx = price_data.idxmax()
    drawdown = (price_data.iloc[max_idx:].min() - price_data.iloc[max_idx]) / price_data.iloc[max_idx]
    drawdown *= 100
    # max drawdown
    for i in range(1, len(price_data.index)):
        curr_drawdown = (price_data.iloc[i] - price_data.iloc[:i].max()) / price_data.iloc[:i].max()
        if curr_drawdown < max_drawdown:
            max_drawdown = curr_drawdown
    max_drawdown *= 100
    if (drawdown == 0):
        drawdown = max_drawdown
    curr_df.loc[0, 'Drawdown'] = drawdown
    curr_df.loc[0, 'Max_Drawdown'] = max_drawdown
    
    # append to df
    max_drawdown_df = pd.concat([max_drawdown_df, curr_df], axis=0, ignore_index=True)       

In [65]:
warnings.resetwarnings()

In [66]:
max_drawdown_df

,Date,Max_Drawdown,Drawdown
0,2017-01-02 00:00:00,-0.00677,-0.00677
1,2017-01-03 00:00:00,-0.002944,-0.000606
2,2017-01-04 00:00:00,-0.001415,-0.001415
3,2017-01-05 00:00:00,-0.001662,-0.001662
4,2017-01-06 00:00:00,-0.000847,-0.000189
...,...,...,...
241,2017-12-22 00:00:00,-0.001412,-0.000388
242,2017-12-26 00:00:00,-0.000922,-0.000922
243,2017-12-27 00:00:00,-0.006471,-0.006471
244,2017-12-28 00:00:00,-0.001118,-0.001118


In [67]:
max_drawdown_df.to_csv('drawdowns.csv')

Now, the drawdown_df is exported as csv and final formatting is again performed using Excel. This marked the end of generating the SampleTradeReport, as all columns were filled.

# This is the end of all code. The Report and Performance Stats were finally compiled using Excel workbook and exported as csv. The equity curve required is present in this notebook itself.